# Cadastro de Localidades Brasileiras Selecionadas do IBGE
## Brazilian Cities Register from IBGE
##### By Jorge Messa Junior

In [18]:
# import pykml to treat kml files
from pykml import parser

# import weg to get file from web
import wget

import pandas as pd

In [15]:
# this is where we can find information about cities in Brazil (check if the url has changed)
url = "ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades/Google_KML/BR_Localidades_2010_v1.kml"
kml_file = wget.download(url)
kml_file_utf = open(kml_file, encoding="utf8")

doc = parser.parse(kml_file_utf)

In [120]:
root = doc.getroot()

schemas = root.Document.Schema

# get the columns structure from the file
# obs Schema8 is the schem where we can find information about cities

schema = "Schema8"
columns = []
for sch in schemas:
    dict_id = sch.attrib
    if dict_id["id"] == schema:
        fields = sch.SimpleField
        for fld in fields:
            columns.append(fld.attrib["name"])
            
print(len(columns))

21


In [121]:
folders = root.Document.Folder

data = []
for fol in folders:
    placemarks = fol.Placemark
    for plc in placemarks:
        sch = plc.ExtendedData.SchemaData
        dict_url= sch.attrib
        if dict_url["schemaUrl"] == ('#'+schema):
            children = sch.getchildren()
            data_row = []
            for child in children:
                
                data_row.append(child.text)
            data.append(data_row)    
            
            
brazil_cities = pd.DataFrame(data, columns=columns)

In [122]:
brazil_cities.head()

,ID,CD_GEOCODIGO,TIPO,CD_GEOCODBA,NM_BAIRRO,CD_GEOCODSD,NM_SUBDISTRITO,CD_GEOCODDS,NM_DISTRITO,CD_GEOCODMU,...,NM_MICRO,NM_MESO,NM_UF,CD_NIVEL,CD_CATEGORIA,NM_CATEGORIA,NM_LOCALIDADE,LONG,LAT,ALT
0,19249,430187505000001,URBANO,None,None,43018750500,None,430187505,BARRA DO QUARAÍ,4301875,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,BARRA DO QUARAÍ,-57.5570603248122,-30.2110754079217,42.0408
1,20512,432240005000001,URBANO,432240005001,Centro,43224000500,None,432240005,URUGUAIANA,4322400,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,URUGUAIANA,-57.0818249090229,-29.7598231720144,78.23019
2,20041,431530505000001,URBANO,None,None,43153050500,None,431530505,QUARAÍ,4315305,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,QUARAÍ,-56.4536470403836,-30.3828679608813,118.674261
3,20167,431710305000001,URBANO,431710305001,Centro,43171030500,None,431710305,SANT'ANA DO LIVRAMENTO,4317103,...,CAMPANHA CENTRAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,SANT'ANA DO LIVRAMENTO,-55.5348142679597,-30.8893840112129,200.677824
4,19734,431060305000001,URBANO,431060305001,Centro,43106030500,None,431060305,ITAQUI,4310603,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,ITAQUI,-56.5571334970304,-29.1286368990609,62.084645


In [123]:
# run if you want to save data in a excel file
brazil_cities.to_excel("brazil_cities_register.xlsx")

# this document is free to share